In [57]:
# import libraries
import sys
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline            

### Provide filename
The file must contain 1 datapoint per line in format (( height, diameter, weight, hue ), label ) which is similar to the format provided for the assignment

# 2 a)

In [6]:
# provide filename
filename = str(input('enter file containing training data: '))

enter file containing training data: 2_a_train.txt


### Read data from file
The function 'create_df' takes a filename as input and returns a pandas dataframe with features and columns and lablels as the last column

In [44]:
def create_df(filename):
    try:
        f = open(filename, 'r')
    except OSError:
        print(f'{filename} could not be opened.\n')
        sys.exit()
        
    # initialize list to store feature and labels for training data
    features = []             
    labels = []
    
    with f:
        line = f.readline()
        while line != '':
            # strip newline and outer parenthesis
            line = line.strip('\n')
            line = line.strip('( )')
            
            # extrace label and append to labels list
            single_label = line.split('), ')[-1]
            labels.append(single_label)
            
            # extrace features and append to features list
            feat = line.split('), ')[0].split(', ')
            features.append(feat)
            
            # read next line
            line = f.readline()
        
        # convert labels list to series
        labels_ser = pd.Series(labels, name = 'label')
        
        # create dataframe of features and append labels
        df = pd.DataFrame(features, columns = ['height', 'diameter', 'weight', 'hue'], dtype = float)
        df['label'] = labels
        
        return df

In [45]:
# create dataframe
df_train = create_df(filename)

In [46]:
# check dataframe head
df_train.head()

,height,diameter,weight,hue,label
0,0.087387,0.060082,0.319795,2.849626,Ceramic
1,0.109784,0.091370,0.473875,3.659025,Metal
2,0.085536,0.030000,0.119153,2.210429,Metal
3,0.084891,0.052961,0.196126,2.707810,Metal
4,0.061071,0.056669,0.246577,4.175536,Ceramic


In [47]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   height    12 non-null     float64
 1   diameter  12 non-null     float64
 2   weight    12 non-null     float64
 3   hue       12 non-null     float64
 4   label     12 non-null     object 
dtypes: float64(4), object(1)
memory usage: 608.0+ bytes


### K Nearest Neighbors

In [77]:
# this function returns the distance and labels of k nearest neighbors of prediction datapoint p
def get_k_nearest_neighbors(df, p, k = 5):
    df_f = df.drop(labels = ['label'], axis = 1)
    
    # get number of datapoints and features
    n, f_n = df_f.shape
    
    sum_of_squared_diff = np.zeros(n)
    
    # use vectorization to sum of squared difference
    for i in range(f_n):
        sum_of_squared_diff = sum_of_squared_diff + (p[i] - df_f.iloc[:,i])**2
    
    # take sq root to get array of cartesian distance
    cartesian_dist = np.sqrt(sum_of_squared_diff)
    
    # create distance series and append with corresponding label
    cartesian_dist_ser = pd.Series(cartesian_dist, name = 'cartesian_distance')
    df = pd.concat([cartesian_dist_ser, df['label']], axis = 1)
    
    df.sort_values(by = 'cartesian_distance', ascending = True, inplace = True)
    
    return df

### Provide Single Datapoint for Prediction

In [24]:
h_p = float(input('Enter height: '))
d_p = float(input('Enter diameter: '))
w_p = float(input('Enter weight: '))
c_p = float(input('Enter hue/color: '))

Enter height: 0.126710477
Enter diameter: 0.068040454
Enter weight: 0.208598827
Enter hue/color: 3.958791026


In [25]:
# save prediction datapont as a 1-D numpy array
p = np.array([h_p, d_p, w_p, c_p])

In [78]:
get_k_nearest_neighbors(df_train, p, 5)

,cartesian_distance,label
4,0.229910,Ceramic
1,0.401325,Metal
9,0.464142,Plastic
6,0.493052,Plastic
10,0.523606,Plastic
5,0.547976,Plastic
7,0.871031,Plastic
8,0.880512,Plastic
11,0.881818,Ceramic
0,1.115446,Ceramic
